# Call packages and libraries

In [12]:
import os
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
from itertools import product
import matplotlib.pyplot as plt

In [13]:
import InstanceGeneratorKP as ig
import InstanceReaderKP as ir
import IPModelKP as ip
import LPModelKP as lp

In [14]:
parent_dir = os.path.dirname(os.getcwd())

# Generate instances if necessary

In [15]:
# Define range of instance parameters
nitems = [50]
n_replications = 10
np.random.seed(0)
seed = list(np.random.choice(range(1000), n_replications, replace=False))
print('Number of instances = ', len(nitems)*n_replications)

Number of instances =  10


In [16]:
# Generate instances
answer = input('Generate instances? (y/n) ')
if answer != 'y':
    print('Instances not generated')
else:
    instance_path = os.path.join(parent_dir, 'Instances', str(time.time()))
    os.makedirs(instance_path, exist_ok=True)
    print('Generating instances...')
    for args in tqdm(product(*[nitems, seed])):
        ig.Instance(*args).write_instance(instance_path)
    print('Instances generated')

Instances not generated


# Solve instances (IP solve)

In [17]:
folder = '1701370803.9812088'
instance_dir = os.path.join(parent_dir, 'Instances', folder)
models_dir = instance_dir.replace('Instances', 'Models_IP')

In [18]:
# Solve instances
answer = input('Solve instances? (y/n) ')
if answer != 'y':
    print('Instances not solved')
else:
    for dir in ['mps', 'lp', 'sol', 'json']:
        os.makedirs(os.path.join(models_dir, dir), exist_ok=True)
    print('Solving instances...')
    for instance in tqdm(os.listdir(instance_dir)):
        if 'instance' in instance:
            capacity, items = ir.Instance(instance_dir, instance).read_instance()
            ip.ModelIP(instance, capacity, items).solve(models_dir)
    print('Instances solved')

Instances not solved


# Solve instances (LP solve)

In [19]:
folder = '1701370803.9812088'
instance_dir = os.path.join(parent_dir, 'Instances', folder)
models_dir = instance_dir.replace('Instances', 'Models_LP')

In [20]:
# Solve instances
answer = input('Solve instances? (y/n) ')
if answer != 'y':
    print('Instances not solved')
else:
    for dir in ['mps', 'lp', 'sol', 'json']:
        os.makedirs(os.path.join(models_dir, dir), exist_ok=True)
    print('Solving instances...')
    for instance in tqdm(os.listdir(instance_dir)):
        if 'instance' in instance:
            capacity, items = ir.Instance(instance_dir, instance).read_instance()
            lp.ModelLP(instance, capacity, items).solve(models_dir)
    print('Instances solved')

Instances not solved


# Load results

In [58]:
df = pd.DataFrame(columns=['Instance', 'n', 'b', 'objVal_IP', 'RHS_IP', 'objVal_LP', 'RHS_LP', 'obj_Gap (%)', 'RHS_Gap (%)'])
for i, instance in enumerate(os.listdir(instance_dir)):
    if 'instance' in instance:
        capacity, items = ir.Instance(instance_dir, instance).read_instance()
        # Solve as IP
        model_ip = ip.ModelIP(instance, capacity, items).load(models_dir)
        used_items_ip = [i for i in items if model_ip.x[i-1] > 0]
        current_capacity_ip = sum([items[i]['weight'] for i in used_items_ip])
        # Solve as LP
        model_lp = lp.ModelLP(instance, capacity, items).load(models_dir)
        objVal_lp = model_lp.objVal
        used_items_lp = [i for i in items if model_lp.x[i-1] > 0.5]
        current_capacity_lp = sum([items[i]['weight'] for i in used_items_lp])
        # LP feasibility check
        c_a_ratio = {i: items[i]['profit']/items[i]['weight'] for i in items} # profit to weight ratio
        c_a_ratio = dict(sorted(c_a_ratio.items(), key=lambda item: item[1])) # sort items by profit to weight ratio
        while current_capacity_lp > capacity:
            for i in c_a_ratio: # remove items with lowest profit to weight ratio until capacity is satisfied
                if i in used_items_lp:
                    used_items_lp.remove(i)
                    objVal_lp -= items[i]['profit']
                    current_capacity_lp -= items[i]['weight']
                    break
        # Store results
        df.loc[i] = [*map(lambda x: int(x), [i, len(items), capacity, 
                                             model_ip.objVal, current_capacity_ip, 
                                             objVal_lp, current_capacity_lp, 0, 0])]
# Format output
df.sort_values(by=['b', 'objVal_IP'], inplace=True)
df.reset_index(drop=True, inplace=True)
df['Instance'] = 1+df.index

In [59]:
df

,Instance,n,b,objVal_IP,RHS_IP,objVal_LP,RHS_LP,obj_Gap (%),RHS_Gap (%)
0,1,50,10,45,10,38,8,0,0
1,2,50,10,55,10,55,9,0,0
2,3,50,10,63,10,63,10,0,0
3,4,50,13,52,13,52,13,0,0
4,5,50,14,62,14,62,13,0,0
5,6,50,16,69,16,69,16,0,0
6,7,50,17,69,17,69,15,0,0
7,8,50,19,66,19,59,17,0,0
8,9,50,19,83,19,83,19,0,0
9,10,50,20,94,20,85,17,0,0
